<a href="https://colab.research.google.com/github/MuhammadIrzam447/NewEncodings/blob/main/Train_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notebook for the training and evalution of ViT for Multi-Class & Multi-Label Dataset i.e MM-IMDB

# Read Training Dataset

In [1]:
import os

image_file_paths = []
image_genre_labels = []
enc_file_paths = []
enc_genre_labels = []

labels_file = "/content/MMLearning/data/imdb/mulitmodal/train_label.txt"
image_folder_add = "/content/MMLearning/data/imdb/mulitmodal/train"

with open(labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')
        image_path = os.path.join(image_folder_add, filename)

        if filename.endswith("_4.png"):
            enc_file_paths.append(image_path)
            enc_genre_labels.append(labels)

        elif filename.endswith("_3.png"):
            image_file_paths.append(image_path)
            image_genre_labels.append(labels)

In [2]:
len(enc_file_paths), len(image_file_paths)

(15552, 15552)

In [3]:
from collections import defaultdict

label_counts = defaultdict(int)

for labels in image_genre_labels:
    for label in labels:
        label_counts[label] += 1

label_count_list = [(label, count) for label, count in label_counts.items()]
sorted_label_count_list = sorted(label_count_list, key=lambda x: x[1], reverse=True)
for label, count in sorted_label_count_list:
    print(f"{label}: {count}")

print("Total Labels: ", len(label_count_list))

Drama: 8424
Comedy: 5108
Romance: 3226
Thriller: 3113
Crime: 2293
Action: 2155
Adventure: 1611
Horror: 1603
Documentary: 1234
Mystery: 1231
Sci-Fi: 1212
Fantasy: 1162
Family: 978
War: 806
Biography: 788
History: 680
Music: 634
Animation: 586
Musical: 503
Western: 423
Sport: 379
Short: 281
Film-Noir: 202
News: 39
Talk-Show: 2
Reality-TV: 1
Total Labels:  26


In [4]:
min_label_count = 40
valid_labels = [label for label, count in label_counts.items() if count >= min_label_count]
valid_labels = sorted(list(valid_labels))

In [5]:
filtered_image_paths = []
filtered_genre_labels = []

for image_path, labels in zip(image_file_paths, image_genre_labels):
    valid_labels_for_sample = [label for label in labels if label in valid_labels]

    if valid_labels_for_sample:
        filtered_image_paths.append(image_path)
        filtered_genre_labels.append(valid_labels_for_sample)

In [6]:
filtered_enc_paths = []
filtered_enc_labels = []

for image_path, labels in zip(enc_file_paths, enc_genre_labels):
    valid_labels_for_sample = [label for label in labels if label in valid_labels]

    if valid_labels_for_sample:
        filtered_enc_paths.append(image_path)
        filtered_enc_labels.append(valid_labels_for_sample)

In [7]:
image_file_paths = filtered_image_paths
image_genre_labels = filtered_genre_labels
enc_file_paths = filtered_enc_paths
enc_genre_labels = filtered_enc_labels

In [8]:
len(image_file_paths), len(enc_file_paths)

(15552, 15552)

In [9]:
import random

combined_data = list(zip(enc_file_paths, enc_genre_labels))
sample_size = int(0.3 * len(combined_data))
sampled_data = random.sample(combined_data, sample_size)


sampled_file_paths, sampled_genre_labels = zip(*sampled_data)

In [10]:
len(sampled_file_paths)

4665

In [11]:
sampled_file_paths = list(sampled_file_paths)
sampled_genre_labels = list(sampled_genre_labels)

In [12]:
train_file_paths = image_file_paths + sampled_file_paths
train_genre_labels = image_genre_labels + sampled_genre_labels

In [13]:
len(train_file_paths), len(train_genre_labels)

(20217, 20217)

# Read Test Dataset

In [14]:
import os

test_image_file_paths = []
test_image_genre_labels = []
test_enc_file_paths = []
test_enc_genre_labels = []

test_labels_file = "/content/MMLearning/data/imdb/mulitmodal/test_label.txt"
image_folder_add = "/content/MMLearning/data/imdb/mulitmodal/test"

with open(test_labels_file, 'r') as file:
    for line in file:
        parts = line.strip().split('|')
        filename = parts[0].strip()
        labels = parts[1].strip().split(', ')
        image_path = os.path.join(image_folder_add, filename)

        if filename.endswith("_4.png"):
            test_enc_file_paths.append(image_path)
            test_enc_genre_labels.append(labels)

        elif filename.endswith("_3.png"):
            test_image_file_paths.append(image_path)
            test_image_genre_labels.append(labels)

In [15]:
len(test_enc_file_paths), len(test_image_file_paths)

(7799, 7799)

In [16]:
test_enc_file_paths[0:2]

['/content/MMLearning/data/imdb/mulitmodal/test/0078718_4.png',
 '/content/MMLearning/data/imdb/mulitmodal/test/0089003_4.png']

In [17]:
test_filtered_image_paths = []
test_filtered_genre_labels = []

for image_path, labels in zip(test_image_file_paths, test_image_genre_labels):
    valid_labels_for_sample = [label for label in labels if label in valid_labels]

    if valid_labels_for_sample:
        test_filtered_image_paths.append(image_path)
        test_filtered_genre_labels.append(valid_labels_for_sample)

In [18]:
test_filtered_enc_paths = []
test_filtered_enc_labels = []

for image_path, labels in zip(test_enc_file_paths, test_enc_genre_labels):
    valid_labels_for_sample = [label for label in labels if label in valid_labels]

    if valid_labels_for_sample:
        test_filtered_enc_paths.append(image_path)
        test_filtered_enc_labels.append(valid_labels_for_sample)

In [19]:
test_image_file_paths = test_filtered_image_paths
test_image_genre_labels = test_filtered_genre_labels
test_enc_file_paths = test_filtered_enc_paths
test_enc_genre_labels = test_filtered_enc_labels

In [20]:
len(test_image_genre_labels), len(test_enc_genre_labels)

(7799, 7799)

In [21]:
import random

test_combined_data = list(zip(test_enc_file_paths, test_enc_genre_labels))
test_sample_size = int(0.3 * len(test_combined_data))
sampled_data = random.sample(test_combined_data, test_sample_size)


test_sampled_file_paths, test_sampled_genre_labels = zip(*sampled_data)

In [22]:
test_sampled_file_paths = list(test_sampled_file_paths)
test_sampled_genre_labels = list(test_sampled_genre_labels)

In [23]:
test_file_paths = test_image_file_paths + test_sampled_file_paths
test_genre_labels = test_image_genre_labels + test_sampled_genre_labels

# ViT and Label Encoding

In [24]:
import torch
!pip install transformers evaluate datasets
import requests
import torch
from PIL import Image
from transformers import *
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memory_efficient_attentio

In [25]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/vit-base-patch16-224"
image_processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name).to(device)

loading configuration file preprocessor_config.json from cache at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3/preprocessor_config.json
size should be a dictionary on of the following set of keys: ({'width', 'height'}, {'shortest_edge'}, {'longest_edge', 'shortest_edge'}, {'longest_edge'}), got 224. Converted to {'height': 224, 'width': 224}.
Image processor ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3/config.json
Model config ViTConfig {
  "_name_or_path

In [26]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit(train_genre_labels)

MultiLabelBinarizer()

In [27]:
transformed_train_genre_labels = mlb.transform(train_genre_labels)

In [28]:
labeling_scheme = mlb.classes_
print(len(labeling_scheme))

23


In [29]:
label_to_class = {i: label for i, label in enumerate(labeling_scheme)}

In [30]:
print("Labeling scheme (binary representations): ", labeling_scheme)
print("Labeling scheme (class names): ", label_to_class)

Labeling scheme (binary representations):  ['Action' 'Adventure' 'Animation' 'Biography' 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Family' 'Fantasy' 'Film-Noir' 'History' 'Horror'
 'Music' 'Musical' 'Mystery' 'Romance' 'Sci-Fi' 'Short' 'Sport' 'Thriller'
 'War' 'Western']
Labeling scheme (class names):  {0: 'Action', 1: 'Adventure', 2: 'Animation', 3: 'Biography', 4: 'Comedy', 5: 'Crime', 6: 'Documentary', 7: 'Drama', 8: 'Family', 9: 'Fantasy', 10: 'Film-Noir', 11: 'History', 12: 'Horror', 13: 'Music', 14: 'Musical', 15: 'Mystery', 16: 'Romance', 17: 'Sci-Fi', 18: 'Short', 19: 'Sport', 20: 'Thriller', 21: 'War', 22: 'Western'}


In [31]:
transformed_train_genre_labels[0]

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0])

In [32]:

mlb1 = MultiLabelBinarizer()
mlb1.fit(test_genre_labels)

MultiLabelBinarizer()

In [33]:
transformed_test_genre_labels = mlb1.transform(test_genre_labels)

In [34]:
labeling_scheme = mlb.classes_

label_to_class = {i: label for i, label in enumerate(labeling_scheme)}

print("Labeling scheme (binary representations): ", labeling_scheme)
print("Labeling scheme (class names): ", label_to_class)


Labeling scheme (binary representations):  ['Action' 'Adventure' 'Animation' 'Biography' 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Family' 'Fantasy' 'Film-Noir' 'History' 'Horror'
 'Music' 'Musical' 'Mystery' 'Romance' 'Sci-Fi' 'Short' 'Sport' 'Thriller'
 'War' 'Western']
Labeling scheme (class names):  {0: 'Action', 1: 'Adventure', 2: 'Animation', 3: 'Biography', 4: 'Comedy', 5: 'Crime', 6: 'Documentary', 7: 'Drama', 8: 'Family', 9: 'Fantasy', 10: 'Film-Noir', 11: 'History', 12: 'Horror', 13: 'Music', 14: 'Musical', 15: 'Mystery', 16: 'Romance', 17: 'Sci-Fi', 18: 'Short', 19: 'Sport', 20: 'Thriller', 21: 'War', 22: 'Western'}


In [35]:
transformed_train_genre_labels[0]

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0])

Create Dataset objects

In [36]:
from datasets import Dataset

train_data = {'image': train_file_paths, 'label': transformed_train_genre_labels}
ds_train = Dataset.from_dict(train_data)

In [37]:
val_data = {'image': test_file_paths, 'label': transformed_test_genre_labels}
ds_val = Dataset.from_dict(val_data)

In [38]:
import PIL.Image as pil

def transform(examples):
  inputs = image_processor([pil.open(img).convert("RGB") for img in examples["image"]], return_tensors="pt")
  inputs["labels"] = examples["label"]
  return inputs

In [39]:
train_dataset = ds_train.with_transform(transform)
val_dataset = ds_val.with_transform(transform)

In [40]:
for item in val_dataset:
  print(item['pixel_values'].shape)
  print(item["labels"])
  break

torch.Size([3, 224, 224])
[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]


In [41]:
import torch

def collate_fn(batch):
  return {
      "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
      "labels": torch.tensor([x["labels"] for x in batch]),
      # "labels": torch.stack([x["labels"] for x in batch], dim=0),
  }

In [42]:
# load the ViT model
model = ViTForImageClassification.from_pretrained(
    model_name,
    num_labels= len(labeling_scheme),
    label2id = {label: str(i) for i, label in enumerate(labeling_scheme)},
    id2label = {str(i): label for i, label in enumerate(labeling_scheme)},
    problem_type = "multi_label_classification",
    ignore_mismatched_sizes=True,
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "Action",
    "1": "Adventure",
    "10": "Film-Noir",
    "11": "History",
    "12": "Horror",
    "13": "Music",
    "14": "Musical",
    "15": "Mystery",
    "16": "Romance",
    "17": "Sci-Fi",
    "18": "Short",
    "19": "Sport",
    "2": "Animation",
    "20": "Thriller",
    "21": "War",
    "22": "Western",
    "3": "Biography",
    "4": "Comedy",
    "5": "Crime",
    "6": "Documentary",
    "7": "Drama",
    "8": "Family",
    "9": "Fantasy"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "interm

In [ ]:
# import torch
# from evaluate import load
# from sklearn.metrics import accuracy_score, f1_score

# accuracy = load("accuracy")
# f1 = load("f1")

# def compute_metrics(eval_pred):
#     raw_logits = torch.tensor(eval_pred.predictions)
#     sigmoid_predictions = torch.sigmoid(raw_logits).numpy()

#     thresholded_predictions = (sigmoid_predictions > 0.5).astype(int)

#     label_ids = eval_pred.label_ids

#     accuracy_score = accuracy.compute(predictions=thresholded_predictions, references=label_ids)
#     f1_score = f1.compute(predictions=thresholded_predictions, references=label_ids, average="macro")

#     return {**accuracy_score, **f1_score}


In [ ]:
# from transformers import TrainingArguments

# training_args = TrainingArguments(
#   output_dir="/content/Model/Models-Train-32", # output directory
#   per_device_train_batch_size=32, # batch size per device during training
#   evaluation_strategy="steps",    # evaluation strategy to adopt during training
#   num_train_epochs=20,             # total number of training epochs
#   # fp16=True,                    # use mixed precision
#   save_steps=10,                # number of update steps before saving checkpoint
#   eval_steps=10,                # number of update steps before evaluating
#   logging_steps=10,             # number of update steps before logging
#   # save_steps=50,
#   # eval_steps=50,
#   # logging_steps=50,
#   save_total_limit=2,             # limit the total amount of checkpoints on disk
#   remove_unused_columns=False,    # remove unused columns from the dataset
#   push_to_hub=False,              # do not push the model to the hub
#   report_to='tensorboard',        # report metrics to tensorboard
#   load_best_model_at_end=True,    # load the best model at the end of training
# )

In [ ]:
# from transformers import Trainer

# trainer = Trainer(
#     model=model,                        # the instantiated 🤗 Transformers model to be trained
#     args=training_args,                 # training arguments, defined above
#     data_collator=collate_fn,           # the data collator that will be used for batching
#     compute_metrics=compute_metrics,    # the metrics function that will be used for evaluation
#     train_dataset=train_dataset,        # training dataset
#     eval_dataset=val_dataset,           # evaluation dataset
#     tokenizer=image_processor,          # the processor that will be used for preprocessing the images
# )

In [ ]:
# trainer.train()

In [ ]:
train_dataset

In [ ]:
val_dataset

In [ ]:
sample = train_dataset[0]

In [ ]:
sample

Setting Hyperparameters

In [43]:
# Training loop
from torch.utils.tensorboard import SummaryWriter
from torch.optim import AdamW
from torch.utils.data import DataLoader
import torch
import torch.nn.functional as F

batch_size = 32

In [44]:
train_dataset_loader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=True)
valid_dataset_loader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=batch_size, shuffle=False)


In [45]:
optimizer = AdamW(model.parameters(), lr=1e-5)


In [46]:
log_dir = "/content/MMLearning/data/Models/Model-10"
summary_writer = SummaryWriter(log_dir=log_dir)

In [47]:
num_epochs = 40
model = model.to(device)

In [48]:
n_train_steps = num_epochs * len(train_dataset_loader)
n_valid_steps = len(valid_dataset_loader)
current_step = 0

In [49]:
n_train_steps , n_valid_steps

(25280, 317)

In [50]:
# logging, eval & save steps
save_steps = 632 #3000

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np

for epoch in range(num_epochs):

    model.train()
    train_loss = 0
    # progress_bar = tqdm(range(current_step, n_train_steps), "Training", dynamic_ncols=True, ncols=80)

    for batch in train_dataset_loader:
      if (current_step+1) % save_steps == 0:
        print()
        print(f"Validation at step {current_step}...")
        print()

        model.eval()

        predictions, labels = [], []
        valid_loss = 0

        for batch in valid_dataset_loader:
            pixel_values = batch["pixel_values"].to(device)
            label_ids = batch["labels"].to(device).float()

            outputs = model(pixel_values=pixel_values, labels=label_ids)

            loss = outputs.loss
            valid_loss += loss.item()

            logits = outputs.logits.detach().cpu()

            # predictions.extend((logits > 0.5).int().cpu().numpy())
            predictions.extend(F.sigmoid(logits).cpu().numpy())
            labels.extend(label_ids.int().cpu().numpy())

        # eval_prediction = EvalPrediction(predictions=predictions, label_ids=labels)
        # metrics = compute_metrics(eval_prediction)
        print()
        print(f"Epoch: {epoch}, Step: {current_step}, Train Loss: {train_loss / save_steps:.4f}, " +
              f"Valid Loss: {valid_loss / n_valid_steps:.4f}") # , Accuracy: {metrics['accuracy']}, " + f"F1 Score: {metrics['f1']}")
        print()

        # summary_writer.add_scalar("valid_loss", valid_loss / n_valid_steps, global_step=current_step)
        # summary_writer.add_scalar("accuracy", metrics["accuracy"], global_step=current_step)
        # summary_writer.add_scalar("f1", metrics["f1"], global_step=current_step)

        model.save_pretrained(f"/content/MMLearning/data/Models/Model-10/checkpoint-{current_step}")
        # image_processor.save_pretrained(f"/content/Model/Models-Train-32/checkpoint-{current_step}")

        predictions = np.array(predictions)
        threshold = 0.5
        predictions = (predictions > threshold).astype(int)

        # accuracy = accuracy_score(labels, predictions)
        # precision = precision_score(labels, predictions, average='macro')
        # recall = recall_score(labels, predictions, average='macro')
        f1 = f1_score(labels, predictions, average='macro')

        # print(f"Accuracy: {accuracy}")
        # print(f"Precision: {precision}")
        # print(f"Recall: {recall}")
        print(f"F1-Score: {f1}")
        # print(classification_report(labels, predictions))

        model.train()
        train_loss, valid_loss = 0, 0

      pixel_values = batch["pixel_values"].to(device)
      labels = batch["labels"].to(device).float()

      outputs = model(pixel_values=pixel_values, labels=labels)

      loss = outputs.loss
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      loss_v = loss.item()
      train_loss += loss_v

      current_step += 1
      # progress_bar.update(1)
      summary_writer.add_scalar("train_loss", loss_v, global_step=current_step)



Validation at step 631...



Configuration saved in /content/MMLearning/data/Models/Model-10/checkpoint-631/config.json



Epoch: 0, Step: 631, Train Loss: 0.2917, Valid Loss: 0.2579



Model weights saved in /content/MMLearning/data/Models/Model-10/checkpoint-631/pytorch_model.bin


F1-Score: 0.06652698548324817

Validation at step 1263...



...

F1-Score: 0.36723715656139155

Validation at step 8215...



Configuration saved in /content/MMLearning/data/Models/Model-10/checkpoint-8215/config.json



Epoch: 12, Step: 8215, Train Loss: 0.1075, Valid Loss: 0.2318



Model weights saved in /content/MMLearning/data/Models/Model-10/checkpoint-8215/pytorch_model.bin


F1-Score: 0.3703981401622095
